# Многомерная минимизация методом Хука Дживса
Цель работы :
*   Реализовать метод Хука–Дживса для минимизации многомерных функций. 
*   Экспериментально сравнить его с методами первого и второго порядка.

# Метод Хука Дживса
Смысл метода Хука дживса заключаеться в поочередном применении исследующего поиска и посика по шаблону.
Для начала импортируем все нужные библиотеки.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy import linalg
import math

Теперь реализуем метод Хука дживса. 
Сначала функция проводит исследующий поиск по всем координатам, мы получим новую точку с наименьшим значением функции в окрестности. Из полученой информации мы можем узнать в каком направлении убывает функция. Продолжая двигаться в этом напровлении мы можем понять в какую сторону изменяеться функция и найти ее экстремум.

In [ ]:
def hooke_jeeves0(fun, x0, h=10.0, tol=1e-10):
    x = x0
    n = len(x0)
    while True:
        y = x.copy() # x - текущая точка, y - следующая
        while True:
            # Сначала аходим минимум в окресности
            f_min = fun(y)
            for j in range(n):
                y[j] += h
                f = fun(y)
                if (f < f_min):
                    f_min = f
                else:
                    y[j] -= 2*h
                    f = fun(y)
                    if (f < f_min):
                        f_min = f
                    else:
                        y[j] += h
            # Затем определяем направление движения
            if fun(y) < fun(x):
                x, y = y, 2*y - x;
                continue
            if linalg.norm(h) < tol:
                return x
            h /= 2
            break

Затем создадим произвольную функцию от двух переменных и x0 для нее. 
$$f=2\sqrt { \frac{ x^2+y^2}{3}}$$


In [ ]:
x0 = np.array([2, 2])
fun = lambda x: 2*math.sqrt((x[0]**2+x[1]**2)/3)


Попробуем получить минимум методом Хука Дживса

In [ ]:
hooke_jeeves0(fun,x0,len(x0)/2)


array([0, 0])

Результат верный, так как минимум функции достигаеться в точке (0,0)
Попробуем слегка изменить функцию и посмотреть повлияет ли это на результат
$$f=2\sqrt { \frac{ x^2+(y-2)^2}{3}}$$

In [ ]:
fun1 = lambda x: 2*math.sqrt((x[0]**2+(x[1]-2)**2)/3)
hooke_jeeves0(fun1,x0,len(x0)/2)


array([0, 2])

Получаем ожидаемый результат в виде точки (0,2)

# Сравнение с методами первого и второго порядка

In [ ]:
from scipy.optimize import minimize, OptimizeResult, rosen, rosen_der, rosen_hess

В качестве метода первого порядка возьмем градиентный спуск, а второго - BFGS 
Для сравнения методов воспользуемся встроенной в scipy функцией Розенброка
$$f(x, y) = (1-x)^2 + 100(y-x^2)^2$$
ее минимум очевидно находиться в точке (1,1), а сама функция принимает в ней знаечение 0

Реализуем метод градиентного спуска подходящим для вызова через scipy.minimize

In [ ]:
def greedy_min(fun, x0, args=(), maxfev=None, alpha=0.0001,
        maxiter=100000, tol=1e-10, callback=None, **options):
    bestx = x0
    besty = fun(x0)
    funcalls = 1
    niter = 0
    improved = True
    stop = False

    while improved and not stop and niter < maxiter:
        niter += 1
        step = alpha * rosen_der(bestx)
        bestx = bestx - step

        besty = fun(bestx)
        funcalls += 1

        if linalg.norm(step) < tol:
            improved = False
        if callback is not None:
            callback(bestx)
        if maxfev is not None and funcalls >= maxfev:
            stop = True
            break

    return OptimizeResult(fun=besty, x=bestx, nit=niter,
                              nfev=funcalls, success=(niter > 1))



In [ ]:
x0 = np.array([4.0, -4.0])
minimize(rosen, x0, method=greedy_min)

     fun: 3.178542782474229e-05
    nfev: 100001
     nit: 100000
 success: True
       x: array([0.99436668, 0.98874248])

Результат работы приблизился к минимуму функции вплоть до 0.01, однако поиск этим методом усперся в 100000 вызовов функции и оказался крайне неэффективен.

Попробуем получить минимум функции через BFGS

In [ ]:
minimize(rosen, x0, method='BFGS', jac=rosen_der)

      fun: 3.454471515983806e-14
 hess_inv: array([[0.52301833, 1.04972664],
       [1.04972664, 2.11241592]])
      jac: array([-6.08772817e-06,  3.14309623e-06])
  message: 'Optimization terminated successfully.'
     nfev: 74
      nit: 56
     njev: 74
   status: 0
  success: True
        x: array([1.0000001 , 1.00000021])

Методу потребовалось всего 56 итераций для получения значения, с точностью до 7 знака после запятой. В сравнении с градиентныйм спуском этот метод оказался намного быстрее.

Слегка изменим уже написаную функцию для метода Хука дживса, чтоб получить возможность вызывать ее через scipy.minimize

In [ ]:
def hooke_jeeves(fun, x0, h=1.0, tol=1e-10, args=(), callback=None, **options):
    bestx = x0
    besty = fun(x0)
    funcalls = 1
    niter = 0
    improved = True
    stop = False
    
    x = x0
    fx = fun(x)
    n = len(x0)
    while True:
        y = x.copy()
        while True:
            niter += 1
            if callback is not None:
                callback(x)
            fy = fun(y); funcalls += 1
            for j in range(n):
                y[j] += h
                f = fun(y); funcalls += 1
                if (f < fy):
                    fy = f
                else:
                    y[j] -= 2*h
                    f = fun(y); funcalls += 1
                    if (f < fy):
                        fy = f
                    else:
                        y[j] += h
            if fy < fx:
                x, y = y, 2*y - x
                fx = fy
                continue
            if linalg.norm(h) < tol:
                return OptimizeResult(fun=fx, x=x, nit=niter,
                              nfev=funcalls, success=(niter > 1))
            h /= 2
            break

Попробуем получить результат через метод Хука Дживса

In [ ]:
minimize(rosen, x0, method=hooke_jeeves)

     fun: 5.421010862427522e-20
    nfev: 1537
     nit: 339
 success: True
       x: array([1., 1.])

В итоге за 339 итераций мы получили точный результат минимума.

# Вывод
При выборе между методами нулевого, первого и второго порядка стоит вопрос в точности и скорости работы. Если вопрос стоит в точности получаймого значения, то стоит использовать метод Хука Дживса, а если в скорости вычисления - то BFSG.
Не существует идельного метода гарантирующего и лучшую скорость, и максимальную точность.